In [1]:
!python3 --version

Python 3.8.10


## Setup



GPU check

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Install required packages

In [4]:
!pip uninstall gym-jsbsim -y
!pip install stable-baselines3[extra]
!pip install tensorboardX
!pip install gym
!pip install matplotlib

Found existing installation: gym-jsbsim 0.6.7-5-gcbda34d
Uninstalling gym-jsbsim-0.6.7-5-gcbda34d:
  Successfully uninstalled gym-jsbsim-0.6.7-5-gcbda34d


In [5]:
# !git clone https://github.com/asgv123/L2F_Challenge_YashGadhia.git

In [6]:
# %cd L2F_Challenge_YashGadhia/
# !git checkout setpoints

In [7]:
!pip install jsbsim
!pip uninstall gym-jsbsim -y
!python3 setup.py install
!pip install -e ./

jsbsim-1.1.11/aircraft/
jsbsim-1.1.11/aircraft/737/
jsbsim-1.1.11/aircraft/737/737.xml
jsbsim-1.1.11/aircraft/737/INSTALL
jsbsim-1.1.11/aircraft/737/cruise_init.xml
jsbsim-1.1.11/aircraft/737/cruise_steady_turn_init.xml
jsbsim-1.1.11/aircraft/737/reset00.xml
jsbsim-1.1.11/aircraft/A320/
jsbsim-1.1.11/aircraft/A320/A320.xml
jsbsim-1.1.11/aircraft/A320/INSTALL
jsbsim-1.1.11/aircraft/A4/
jsbsim-1.1.11/aircraft/A4/A4.xml
jsbsim-1.1.11/aircraft/A4/INSTALL
jsbsim-1.1.11/aircraft/B17/
jsbsim-1.1.11/aircraft/B17/B17.xml
jsbsim-1.1.11/aircraft/B17/INSTALL
jsbsim-1.1.11/aircraft/B17/reset00.xml
jsbsim-1.1.11/aircraft/B747/
jsbsim-1.1.11/aircraft/B747/B747.xml
jsbsim-1.1.11/aircraft/B747/INSTALL
jsbsim-1.1.11/aircraft/B747/reset00.xml
jsbsim-1.1.11/aircraft/Boeing314/
jsbsim-1.1.11/aircraft/Boeing314/Boeing314.xml
jsbsim-1.1.11/aircraft/Boeing314/INSTALL
jsbsim-1.1.11/aircraft/C130/
jsbsim-1.1.11/aircraft/C130/C130.xml
jsbsim-1.1.11/aircraft/C130/INSTALL
jsbsim-1.1.11/aircraft/C130/reset00.xml
js

In [8]:
import gym
import gym_jsbsim
from gym_jsbsim.catalogs.catalog import Catalog as c
import numpy as np
import math
from stable_baselines3 import DQN, A2C, DDPG, HER, PPO, SAC, TD3
from stable_baselines3.common.env_checker import check_env

## Main code

Wrappers to make observations/actions conform to stable-baselines3 requirements

Reward wrapper  
- Will use MSE(($dr^*$, $d\theta ^*$), ($dr_{obs}$, $d\theta_{obs}$))

Trajectory function
- tries to connect current heading to target heading via a circular path
- provides above path as incremental ($\Delta r$, $Δθ$) 
- input turning time, speed (fps), angle (deg), dt (which is simulation time corresponding to each iteration)

In [9]:
env = gym.make("GymJsbsim-HeadingControlTask-v0")

is_debug is False


In [10]:
check_env(env)



     JSBSim Flight Dynamics Model v1.1.11 [GitHub build 741/commit 10e077b486c42d38e76d1dd7f0aeb744f92d96a0] Feb 13 2022 12:40:33
            [JSBSim-ML v2.0]

JSBSim startup beginning ...



/home/andrew/.local/lib/python3.8/site-packages/stable_baselines3/common/env_checker.py:272: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
/home/andrew/.local/lib/python3.8/site-packages/stable_baselines3/common/env_checker.py:278: UserWarning: Your action space has dtype float64, we recommend using np.float32 to avoid cast errors.
  warnings.warn(

In file /home/andrew/sem8/cs748/L2F_Challenge_YashGadhia/gym_jsbsim/jsbsim-1.1.11/aircraft/A320/A320.xml: line 251
Engine location ignored, only thruster location is used.

In file /home/andrew/sem8/cs748/L2F_Challenge_YashGadhia/gym_jsbsim/jsbsim-1.1.11/aircraft/A320/A320.xml: line 256
Engine orientation ignored, only thruster orientation is used.

In file /home/andrew/sem8/cs748/L2F_Challenge_YashGadhia/gym_jsbsim/jsbsim-1.1.11/aircraft/A320/A320.xml: line 276
Engine location ignored, only thruster l

AssertionError: The observation returned by the `reset()` method should be a single value, not a tuple

In [ ]:
# model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="./ppo_tensorboard/")
# %load_ext tensorboard
# %tensorboard --logdir ./ppo_tensorboard/

# model.learn(total_timesteps=10000000)
# model.save("/content/drive/Shareddrives/Learn_to_fly/ppo_model_setpoints_1")
# del model

In [ ]:
# model = PPO.load("/content/drive/Shareddrives/Learn_to_fly/ppo_model_setpoints_1", env=env)
# episode_reward = 0
# state = env.reset()
# print("Initial State =", state)
# done = False
# while not done:
#   action, _ = model.predict(state, deterministic=True)
#   state, reward, done, _ = env.step(action)
#   episode_reward += reward
#   print("action =", action, " ---> State =", state, " : Reward =", reward)
# print("Simulation Time = "+"{:.3f}".format(env.get_sim_time())+" sec")
# print("Episode Reward = "+"{:.3f}".format(episode_reward))